# PCT_URBAN and urban surface fractions with 3 landunit types
- This file is used to calculate the urban fraction in the inputdata;

In [1]:
import numpy as np
import xarray as xr  
import pandas as pd

In [2]:
path = '/work/n02/n02/yuansun/cesm/'
# surface
sfile = path + 'cesm_inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr1850_c190214.nc'
ds = xr.open_dataset(sfile)
ds

<xarray.Dataset>
Dimensions:                  (lsmlat: 192, lsmlon: 288, nlevsoi: 10,
                              natpft: 15, cft: 2, time: 12, lsmpft: 17,
                              numurbl: 3, numrad: 2, nlevurb: 5, nglcecp1: 11,
                              nglcec: 10)
Coordinates:
  * natpft                   (natpft) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * cft                      (cft) int32 15 16
  * time                     (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lsmlat, lsmlon, nlevsoi, lsmpft, numurbl,
                                numrad, nlevurb, nglcecp1, nglcec
Data variables: (12/77)
    mxsoil_color             int32 ...
    SOIL_COLOR               (lsmlat, lsmlon) int32 ...
    PCT_SAND                 (nlevsoi, lsmlat, lsmlon) float64 ...
    PCT_CLAY                 (nlevsoi, lsmlat, lsmlon) float64 ...
    ORGANIC                  (nlevsoi, lsmlat, lsmlon) float64 ...
    FMAX                     (lsmlat, lsmlon) float64 ...
    ...                       ...
    CONST_HARVEST_SH2        (lsmlat, lsmlon) float64 ...
    CONST_HARVEST_SH3        (lsmlat, lsmlon) float64 ...
    CONST_GRAZING            (lsmlat, lsmlon) float64 ...
    CONST_FERTNITRO_CFT      (cft, lsmlat, lsmlon) float64 ...
    UNREPRESENTED_PFT_LULCC  (natpft, lsmlat, lsmlon) float64 ...
    UNREPRESENTED_CFT_LULCC  (cft, lsmlat, lsmlon) float64 ...
Attributes: (12/48)
    Conventions:                          NCAR-CSM
    History_Log:                          created on: 02-14-19 10:18:15
    Logname:                              erik
    Host:                                 cheyenne3
    Source:                               Community Land Model: CLM5
    Version:                              release-clm5.0.18/gpfs/fs1/scratch/...
    ...                                   ...
    map_topography_stats_file:            map_1km-merge-10min_HYDRO1K-merge-n...
    Soil_texture_raw_data_file_name:      mksrf_soitex.10level.c010119.nc
    Soil_color_raw_data_file_name:        mksrf_soilcolor_CMIP6_simyr2005.c17...
    Fmax_raw_data_file_name:              mksrf_fmax_3x3min_USGS_c120911.nc
    Organic_matter_raw_data_file_name:    mksrf_organic_10level_5x5min_ISRIC-...
    Vegetation_type_raw_data_filename:    mksrf_landuse_histclm50_LUH2_1850.c...

In [7]:
urban_mask = np.any(ds['PCT_URBAN'] != 0, axis=0)

In [8]:
area = ds['AREA'].where(urban_mask)
land = ds['LANDFRAC_PFT'].where(urban_mask)
urban = ds['PCT_URBAN'].where(urban_mask)
roof = ds['WTLUNIT_ROOF'].where(urban_mask)
improad = ((1 - ds['WTLUNIT_ROOF']) * (1 - ds['WTROAD_PERV'])).where(urban_mask)

In [9]:
# global average urban percent
global_urban = area*land*urban
global_tbd = global_urban[:,:,0].sum()/global_urban.sum()
global_hd = global_urban[:,:,1].sum()/global_urban.sum()
global_md = global_urban[:,:,2].sum()/global_urban.sum()
print(global_tbd*100, global_hd*100, global_md*100)

<xarray.DataArray ()>
array(0.09866587) <xarray.DataArray ()>
array(20.88765967) <xarray.DataArray ()>
array(79.01367446)


In [25]:
pct_roof = (area*land*urban*roof)
# pct_roof: lsmlat: 192lsmlon: 288numurbl: 3
global_roof = pct_roof.sum()/(area*land*urban).sum()
global_roof_tbd = pct_roof[:,:,0].sum()/global_urban.sum()
global_roof_hd = pct_roof[:,:,1].sum()/global_urban.sum()
global_roof_md = pct_roof[:,:,2].sum()/global_urban.sum()
print(global_roof*100, global_roof_tbd*100, global_roof_hd*100, global_roof_md*100)

<xarray.DataArray ()>
array(44.33479862) <xarray.DataArray ()>
array(0.06061233) <xarray.DataArray ()>
array(12.84810001) <xarray.DataArray ()>
array(31.42608629)


In [15]:
# roof fractio at the grid level
global_roof_g = (area*land*urban*roof).sum()/(area*land).sum()
print(global_roof_g)

<xarray.DataArray ()>
array(0.86596272)


In [20]:
# urban fraction
print((area*land*urban).sum()/(area*land).sum())

<xarray.DataArray ()>
array(1.95323481)


In [19]:
# 0.01 roof albedo change
print(global_roof_g * 0.01 * 0.01)

<xarray.DataArray ()>
array(8.65962718e-05)


In [29]:
pct_improad = area*land*urban*improad
global_improad = pct_improad.sum()/(area*land*urban).sum()
global_improad_tbd = pct_improad[:,:,0].sum()/global_urban.sum()
global_improad_hd = pct_improad[:,:,1].sum()/global_urban.sum()
global_improad_md = pct_improad[:,:,2].sum()/global_urban.sum()
print(global_improad*100, global_improad_tbd*100, global_improad_hd*100, global_improad_md*100)

<xarray.DataArray ()>
array(15.49935505) <xarray.DataArray ()>
array(0.0287241) <xarray.DataArray ()>
array(3.82813673) <xarray.DataArray ()>
array(11.64249422)
